# Imports

In [1]:
from numpy.random import seed
seed(888)
import tensorflow 
tensorflow.random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf

from time import strftime
from PIL import Image

# Constants

In [3]:
## x - images pixel
## y - actual value

X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digits_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_HEIGHT = 28
IMAGE_WIDTH = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT*IMAGE_WIDTH*CHANNELS

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

Wall time: 175 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

Wall time: 21.1 s


In [8]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

Wall time: 3.68 s


# Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
x_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
y_train_all

array([5, 0, 4, ..., 5, 6, 8])

In [13]:
y_test

array([7, 2, 1, ..., 4, 5, 6])

In [14]:
y_train_all.shape

(60000,)

In [15]:
x_test.shape

(10000, 784)

# Data Preprocessing

In [16]:
## Rescale (easy for neural network to process)
x_train_all, x_test = x_train_all/255.0, x_test/255.0

#### Convert target values to one-hot encoding

In [17]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

In [18]:
values = y_train_all[:5]
np.eye(10)[values]
## no of classes = 10

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [19]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [20]:
y_train_all[:5]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [21]:
y_train_all.shape

(60000, 10)

In [22]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

In [23]:
y_test[:5]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

### Create validation dataset from training data

In [24]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [25]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [26]:
x_train.shape

(50000, 784)

In [27]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [28]:
tf.compat.v1.disable_eager_execution()

In [29]:
## Statring setup for tensor, also used for graph
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X') ## features
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels') ## labels 

## Neural Network Archi

### Hyperparameters 

In [30]:
''' 
Para which we are gonna determine ahead of time. These include how long to 
train the model, no of epoch,learning rate in optimiser, no of layers 
in neural network
'''

' \nPara which we are gonna determine ahead of time. These include how long to \ntrain the model, no of epoch,learning rate in optimiser, no of layers \nin neural network\n'

In [31]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [32]:
## Function for layer
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim,stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W') 

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
            
        return layer_out    

In [33]:
# ## Model without dropout
# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
#                      bias_dim=[n_hidden1], name='layer_1') 

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                      bias_dim=[n_hidden2], name='layer_2') 

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
#                      bias_dim=[NR_CLASSES], name='out') 

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [34]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                     bias_dim=[n_hidden1], name='layer_1') 

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                     bias_dim=[n_hidden2], name='layer_2') 

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                     bias_dim=[NR_CLASSES], name='out') 

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'
## DO - dropout
## LR - learning rate
## E - epoch


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [35]:
## Folder for Tensorboard
folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Directory Created!')

Directory Created!


# Loss, Optimisation & Metrics

#### Defining Loss Function

In [36]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                     logits=output))

#### Defining Optimizer

In [37]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    ## minimize the loss when we iterate over data
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [38]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [39]:
## creating summary
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost_or_loss', loss)

#### Check input images in Tensorboard

In [40]:
## showing how images are in trained model 
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1,28,28,1]) ## (no of images,hei,wid,ch)
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [41]:
sess = tf.Session()
## tensorflow session is when placeholder start getting its value

#### Setup Filewriter & Merge Summaries

In [42]:
## summary = graphs
merged_summary = tf.summary.merge_all()## merge all graphs

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

#### Initialize variables

In [43]:
init = tf.global_variables_initializer()
sess.run(init)

### Batching the Data

In [44]:
size_of_batch = 1000

In [45]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [46]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch
    
    return data[start:end], labels[start:end]    

### Training Loop

In [47]:
for epoch in range(nr_epochs):
    
    ## -----Training Dataset-------
    for i in range(nr_iterations):
        ## returns data & label
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train
                                      ,labels=y_train)
        ## setting the data
        feed_dictionary = {X:batch_x, Y:batch_y}
        ## running the calculations
        sess.run(train_step, feed_dict=feed_dictionary)
    ## minizise loss(train_step) on feed_dictionary & will update the weights
        
    ## Summary at every epoch
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], 
                                  feed_dict=feed_dictionary)
    ## take the summary(s) and write it on disk(epoch)
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch{epoch} \t| Training Accuracy = {batch_accuracy}')
    
    ## -----Validation Dataset-------
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)    

Epoch0 	| Training Accuracy = 0.8510000109672546
Epoch1 	| Training Accuracy = 0.8640000224113464
Epoch2 	| Training Accuracy = 0.8709999918937683
Epoch3 	| Training Accuracy = 0.8700000047683716
Epoch4 	| Training Accuracy = 0.871999979019165
Epoch5 	| Training Accuracy = 0.8759999871253967
Epoch6 	| Training Accuracy = 0.9769999980926514
Epoch7 	| Training Accuracy = 0.9769999980926514
Epoch8 	| Training Accuracy = 0.984000027179718
Epoch9 	| Training Accuracy = 0.9819999933242798
Epoch10 	| Training Accuracy = 0.9819999933242798
Epoch11 	| Training Accuracy = 0.984000027179718
Epoch12 	| Training Accuracy = 0.9850000143051147
Epoch13 	| Training Accuracy = 0.9869999885559082
Epoch14 	| Training Accuracy = 0.984000027179718
Epoch15 	| Training Accuracy = 0.9900000095367432
Epoch16 	| Training Accuracy = 0.9890000224113464
Epoch17 	| Training Accuracy = 0.9909999966621399
Epoch18 	| Training Accuracy = 0.9890000224113464
Epoch19 	| Training Accuracy = 0.9860000014305115
Epoch20 	| Tra

# Saving the Model

In [48]:
outputs = {'accuracy_calc/prediction': model_prediction}
inputs = {'X':X}
tf.compat.v1.saved_model.simple_save(session=sess, export_dir='SavedModel', 
                                 inputs=inputs, outputs=outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


# Make predictions

In [49]:
img = Image.open('MNIST/test_img.png')
img

In [50]:
## Convert to grayscale 
bw = img.convert('L')

In [51]:
## Invert image (as model is trained with black bg & white text)
img_array = np.invert(bw)

In [52]:
img_array.shape

(28, 28)

In [53]:
## Flattening img, i.e- (28,28) to (784)
test_img = img_array.ravel()

In [54]:
test_img.shape

(784,)

In [55]:
## Return highest prob for the image
prediction = sess.run(fetches=tf.argmax(output, axis=1), 
                      feed_dict={X:[test_img]})

In [56]:
print(f'Prediction for text img (ans=2) = {prediction}')

Prediction for text img (ans=2) = [2]


# Testing and Evaluation

In [57]:
test_accuracy = sess.run(fetches=accuracy, 
                         feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test dataset = {test_accuracy:0.2%}')

Accuracy on test dataset = 97.82%


# Reset for next run

In [58]:
# train_writer.close()
# validation_writer.close()
# sess.close()
# tf.reset_default_graph()

# Code used in 1st part of module

In [59]:
# ## FOR 1st HIDDEN LAYER
# with tf.name_scope('hidden_1'):
#     ## staring values to weights & bias
#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1],
#                                     stddev=0.1, seed=42)
#     ## truncated means there will not be any extreme values on tail left or right
#     ## shape(no of inputs, no of neurons)

#     ## Variable willl hold weights in the 1st hidden layer
#     w1 = tf.Variable(initial_value=initial_w1, name='w1') 
    
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     ## inputs in for the 1st layer
#     layer1_in = tf.matmul(X, w1) + b1
    
#     layer1_out = tf.nn.relu(layer1_in)
#     ## nn.relu is the activation fn

In [60]:
# '''
# All is setup is for 1st hidden layer, the weights & biases will be upadted
# during the training process. These values will be fed into in the activation
# function of neurons and represent the strength of conections b/w diff units
# '''

In [61]:
# ## FOR 2nd HIDDEN LAYER
# with tf.name_scope('hidden_2'):
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1,n_hidden2],
#                                     stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2') 

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [62]:
# ## FOR 3rd (OUTPUT) LAYER
# with tf.name_scope('output_layer'):

#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES],
#                                     stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3') 

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [63]:
#b3.eval(sess)